## Initialize

In [ ]:
getwd()

In [ ]:
#library(Rmisc)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

dataset_name = "210714_metabolomics"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = glue("{base_path}/data")
dataset_path = glue("{data_path}/3_datasets_post/{dataset_name}")

project_label="21_metabolomics_multitask"
project_path = glue("{base_path}/results/projects/{project_label}")
figures_path = glue("{project_path}/figures")
data_results_path = glue("{project_path}/data")

## Load data

In [ ]:
list.dirs(path = project_path, full.names = TRUE, recursive = TRUE)

In [ ]:
run = "220126"

In [ ]:
DSM = "MultiTaskSurvivalTraining"
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) 
data_description = arrow::read_feather(glue("{dataset_path}/description_merged.feather"))
predictions = arrow::read_feather(glue("{data_results_path}/predictions_{run}_metabolomics.feather")) 

In [ ]:
data_events = data %>% select(eid, ends_with("event"), ends_with("event_time")) %>% 
    pivot_longer(-eid, names_to=c("endpoint", "type"), values_to="value", names_pattern="(.*)(event_time|event)") %>% 
    mutate(endpoint = stringr::str_sub(endpoint, end=-2)) %>% pivot_wider(names_from="type", values_from="value")

In [ ]:
predictions_tte = predictions %>% left_join(data_events, by=c("endpoint", "eid"))

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

#endpoint_order = c("M_type_2_diabetes", "M_all_cause_dementia", "M_heart_failure")
endpoint_order = c("M_MACE", "M_coronary_heart_disease", "M_cerebral_stroke", "M_all_cause_dementia", "M_heart_failure", "M_atrial_fibrillation",
                   "M_type_2_diabetes", "M_liver_disease", "M_renal_disease", "M_peripheral_arterial_disease", "M_venous_thrombosis",  "M_abdominal_aortic_aneurysm",
                   "M_chronic_obstructuve_pulmonary_disease", "M_asthma", 'M_parkinsons_disease', 'M_cataracts', 'M_glaucoma', 'M_fractures',
                    "M_lung_cancer","M_non_melanoma_skin_cancer","M_colon_cancer","M_rectal_cancer","M_prostate_cancer","M_breast_cancer")

In [ ]:
library("jsonlite")
colors_path = "colors.json"
colors_dict = read_json(colors_path)

color_map <- c('COX_Age+Sex' = colors_dict$pastel$grey$light, 
               'DS_Metabolomics' = "#4F8EC1",  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = colors_dict$pastel$grey$dark, 
               'COX_PANEL' = "black", 
               'DS_Age+Sex+Metabolomics' = "forestgreen", #"#53dd6c",#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = "#d8315b",#colors_dict$pastel$red$mid,
               'DS_PANELnoblood+Metabolomics' = "#023768", #"forestgreen",
               'DS_PANEL+Metabolomics' = "black" #colors_dict$pastel$red$dark
      )

type_map <- c('COX_Age+Sex' = "solid", 
               'DS_Metabolomics' = "solid",  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = "dashed", 
               'COX_PANEL' = "dotted", 
               'DS_Age+Sex+Metabolomics' = "solid",#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = "solid",#colors_dict$pastel$red$mid,
               'DS_PANEL+Metabolomics' = "solid", #colors_dict$pastel$red$dark
                'DS_PANELnoblood+Metabolomics' = "solid",
              "Treat All" = "solid",
              "Treat None" = "solid"
      )

size_map <- c('COX_Age+Sex' = 0.25, 
               'DS_Metabolomics' = 0.25,  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = 0.25, 
               'COX_PANEL' = 0.5, 
               'DS_Age+Sex+Metabolomics' = 0.25,#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = 0.25,#colors_dict$pastel$red$mid,
               'DS_PANEL+Metabolomics' = 0.25, #colors_dict$pastel$red$dark
                'DS_PANELnoblood+Metabolomics' = 0.25,
              "Treat All" = 0.25,
              "Treat None" = 0.25
      )

In [ ]:
library(foreach)

In [ ]:
library(doParallel)

In [ ]:
scores_dca = c("DS_Metabolomics",
    "COX_Age+Sex",
    "DS_Age+Sex+Metabolomics" ,
    "COX_ASCVD",
    "DS_ASCVD+Metabolomics",
    "DS_PANELnoblood+Metabolomics",
    "COX_PANEL",
    "DS_PANEL+Metabolomics"
    # "DS_PANELjustbloodcount+Metabolomics",          
    
              )

In [ ]:
prepare_endpoint = function(endpoint){
    dca_temp = predictions_tte %>% filter(endpoint==!!endpoint) %>% 
        mutate(score=glue("{module}_{features}")) %>% 
        select(eid, score, Ft_10, event, event_time) %>% 
        filter(score %in% scores_dca) %>%
        pivot_wider(names_from="score", values_from="Ft_10") %>% 
        distinct() %>% 
        mutate(
            event10 = case_when(
                (event == 1 & event_time <= 10) ~ 1,
                (event == 1 & event_time > 10) ~ 0,
                TRUE ~ 0)
        ) %>%
        mutate(endpoint=endpoint)
    return(dca_temp)# %>% select(-event, -event_time))
    }

In [ ]:
library(pROC)
calc_roc = function(pred_df, pred_col){
    tryCatch(
        {
        pred = pred_df[[pred_col]]#ifelse(pred_df[[pred_col]]>threshold,1,0)
        actual = pred_df$event
        roc_obj = roc(actual, pred)
        x = coords(roc_obj,  ret = c("threshold", "sensitivity", "specificity", "ppv", "npv"))
        roc_df <- data.frame(
        threshold = rev(roc_obj$thresholds),
      sens=rev(roc_obj$sensitivities), 
      spec=rev(roc_obj$specificities),
        ppv = rev(x$ppv)) %>%
        mutate(lpr = sens / (1-spec))
        return (roc_df)
        },
        error = function(e){
            #message("Caugth an error")
            #print(e)
            error_tribble =  tribble(
                ~threshold, ~sens, ~spec, ~lpr, 
                NA, NA, NA, NA)
            return (error_tribble)
        }
        )
        }

In [ ]:
dfs = tibble()
for (endpoint in endpoint_order){
    print(endpoint)
    flush.console()
    df_endpoint = prepare_endpoint(endpoint)
    for (score in scores_dca){
        sens_spec = calc_roc(df_endpoint, score) %>% mutate(endpoint=endpoint, score=score)
        dfs = bind_rows(dfs, sens_spec)
        }
    
    }

In [ ]:
roc_df = dfs %>% filter(!is.infinite(threshold))

In [ ]:
# specificity

In [ ]:
detection_df = roc_df %>% 
    mutate(fpr=1-spec) %>%
    mutate(c_5 = fpr-0.05, c_10 = fpr-0.1, c_15 = fpr-0.15, c_20 = fpr-0.20, c_25 = fpr-0.25, c_30 = fpr-0.3, c_35 = fpr-0.35, c_40 = fpr-0.4, c_45 = fpr-0.45, c_50 = fpr-0.5) %>% 
    pivot_longer(starts_with("c_"), names_to="fpr_name", values_to="c_x") %>%
    group_by(endpoint, score, fpr_name) %>% 
    slice_min(abs(c_x)) %>% select(-c_x) %>% ungroup() %>%
    group_by(endpoint, score, fpr_name) %>% 
    summarise(threshold = mean(threshold), sens=mean(sens), ppv=mean(ppv), lpr=mean(lpr)) %>%
     mutate(fpr = as.numeric(str_remove(fpr_name, "c_"))/100) %>% 
    mutate(score = factor(score, levels= scores_dca)) %>%
    mutate(endpoint = factor(endpoint, levels= endpoint_order)) %>%
    select(-fpr_name) %>% select(endpoint, score, fpr, threshold, sens, ppv, lpr) %>% arrange(endpoint, fpr, score) %>% ungroup()

In [ ]:
scores_map = c(
    "DS_Metabolomics" = "MET",
    
    "COX_Age+Sex" = "AgeSex",
    "DS_Age+Sex+Metabolomics" = "AgeSex+MET",
    
    "COX_ASCVDnoblood" = "ASCVD(noblood)",
    "DS_ASCVDnoblood+Metabolomics" = "ASCVD(noblood)+MET",
    
    "COX_ASCVD" = "ASCVD",
    "DS_ASCVD+Metabolomics" = "ASCVD+MET",
    
     "COX_PANELnoblood" = "PANEL(noB)", 
    "DS_PANELnoblood+Metabolomics" = "PANEL(noB)+MET",
    
    "COX_PANELjustbloodcount" = "PANEL(jBC)", 
    "DS_PANELjustbloodcount+Metabolomics" = "PANEL(jBC)+MET",
    
    "COX_PANEL" = "PANEL", 
    "DS_PANEL+Metabolomics" = "PANEL+MET"
               )

In [ ]:
base_size = 8
title_size = 8
facet_size = 8
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=8), axis.text=element_text(size=8, color="black"), axis.text.x=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2)))

In [ ]:
plot_width=8.25; plot_height=5.5; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)

library(ggdist)

temp = detection_df %>% filter(fpr==0.05)
met_lpr = ggplot(temp, aes(x=score)) +
    labs(x=NULL, y="Positive Likelyhood Ratio (at 5% FPR)")+#, title="Metabolomics (orange) vs. Age+Sex (grey) vs. AgeSexMetabolomic (red)")+
    geom_point(mapping = aes(y=lpr, color=score, fill=score), alpha=0.7, size=0.7)+
    scale_x_discrete(labels=scores_map)+
    scale_y_continuous(breaks=scales::extended_breaks())+
    scale_color_manual(values=color_map)+scale_fill_manual(values=color_map)+
    #coord_flip() + 
    facet_wrap(~endpoint, scales="free_y", labeller = labeller(endpoint = endpoint_map), ncol=6) +theme(legend.position="none")+theme(axis.text.x = element_text(angle =90, hjust = 1))+#+#+
    theme(panel.spacing = unit(0.8, "lines"), 
          panel.grid.major.y = element_line(), 
          axis.text.x = element_text(size=5.5, hjust=1))

met_lpr

In [ ]:
table_nice = detection_df %>% 
    mutate(endpoint = recode(endpoint, !!!endpoint_map), score = recode(score, !!!scores_map)) %>% arrange(endpoint, score)
table_nice %>% write_csv("outputs/Suppl_Table_PPV.csv")

In [ ]:
table_nice

In [ ]:
table_wide = table_nice %>% rename(`1_threshold`=threshold, `2_sens`=sens, `3_ppv`=ppv, `4_lpr`=lpr) %>%
    pivot_wider(names_from="fpr", values_from=c("1_threshold", "2_sens", "3_ppv", "4_lpr"), names_glue="{fpr}_{.value}") %>% select(sort(colnames(.))) %>% select(endpoint, score, everything())
table_wide %>% write_csv("outputs/Suppl_Table_PPV_wide.csv")

In [ ]:
table_wide

In [ ]:
detection_df_clean = detection_df %>% select(ppv, endpoint, score, fpr) %>% 
    pivot_wider(names_from="score", values_from="ppv", values_fn=mean) %>% 
    arrange(endpoint, fpr)

In [ ]:
detection_df_clean

In [ ]:
plot_width=15; plot_height=5; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
ggplot(detection_df, aes(x=fpr, y=ppv, color=score, linetype=score, size=score)) + geom_line() + 
    facet_wrap(~endpoint, scales="free_y") +
  scale_color_manual(values=color_map)+
    scale_linetype_manual(values=type_map)+
    scale_size_manual(values=size_map)

In [ ]:
plot_width=15; plot_height=5; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
ggplot(detection_df, aes(x=spec, y=sens, color=score, linetype=score, size=score)) + geom_line() + facet_grid(~endpoint) +
  scale_color_manual(values=color_map)+
    scale_linetype_manual(values=type_map)+
    scale_size_manual(values=size_map)

In [ ]:
plot_width=15; plot_height=5; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
ggplot(dfs, aes(x=threshold, y=lpr, color=score, linetype=score, size=score)) + geom_line() + facet_grid(~endpoint) +
  scale_color_manual(values=color_map)+
    scale_linetype_manual(values=type_map)+
    scale_size_manual(values=size_map)
    
    

In [ ]:
roc_df_long = dfs %>% pivot_longer(c(sens, spec), names_to="type", values_to="value")

In [ ]:
plot_width=8.25; plot_height=10; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
ggplot(roc_df_long, aes(x=threshold, y=value, color=type)) + geom_line() + facet_grid(score~endpoint)

In [ ]:
temp = dfs %>% mutate(lpr = Sensitivity/(1-Specificity))

In [ ]:
temp 

In [ ]:
temp %>% 
    select(endpoint, score, threshold, lpr) %>% 
    pivot_wider(names_from="score", values_from="lpr") %>% 
    select(endpoint, threshold, all_of(scores_dca)) 

In [ ]:
plot_width=8.25; plot_height=10; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
ggplot(temp, aes(x=threshold, y=lpr, color=score, type=score, size=score)) + 
    geom_line() + 
    scale_color_manual(values=color_map)+
    scale_linetype_manual(values=type_map)+
    scale_size_manual(values=size_map)+
    facet_grid(endpoint~., scales="free_y") 

In [ ]:
calc_plr(df_endpoint, "DS_Metabolomics", threshold)

In [ ]:
endpoint = "M_type_2_diabetes"


In [ ]:
threshold = 0.1
calc_plr(df_endpoint, "DS_Metabolomics", threshold)

In [ ]:
tribble(
    ~Sensitivity, ~Specificity, 
    sensitivity(conf_matrix), specificity(conf_matrix)
)
    

In [ ]:
sensitivity(conf_matrix)

In [ ]:
specificity(conf_matrix)

In [ ]:
plr = sensitivity(conf_matrix) / (1-specificity(conf_matrix))
plr

In [ ]:
temp

In [ ]:
library(ROCit)

In [ ]:
survivalROC(temp$event_time, temp$event, temp$Ft, predict.time=10)

In [ ]:
calculate_dca = function(endpoint){
    dca_temp = predictions_tte %>% filter(endpoint==!!endpoint) %>% 
        mutate(score=glue("{module}_{features}")) %>% 
        select(eid, score, Ft_10, event, event_time) %>% 
        filter(score %in% scores_dca) %>%
        pivot_wider(names_from="score", values_from="Ft_10") %>% 
        distinct()
    
    print(glue("{endpoint} n={nrow(dca_temp)}"))
    flush.console()
    
    dca_endpoint = dca(
        Surv(event_time, event) ~ DS_Metabolomics + `COX_Age+Sex` + `DS_Age+Sex+Metabolomics` + COX_ASCVD + `DS_ASCVD+Metabolomics` + COX_PANEL + `DS_PANEL+Metabolomics` + `DS_PANELjustbloodcount+Metabolomics` + `DS_PANELnoblood+Metabolomics`, 
        data = dca_temp,
        time = 10,
        thresholds = seq(0, 0.5, by=0.001)
           )$dca
    return(dca_endpoint %>% mutate(st_net_benefit = net_benefit/prevalence))
}

In [ ]:
registerDoParallel(length(endpoint_order))

In [ ]:
library(dcurves)
dca_data = foreach(key=endpoint_order, endpoint=endpoint_order) %dopar% {
    calculate_dca(endpoint)
}

In [ ]:
names(dca_data) = endpoint_order
for (endpoint in endpoint_order){
    dca_data[[endpoint]]["endpoint"] = endpoint
    }

In [ ]:
dca_data_agg = bind_rows(dca_data) 

In [ ]:
dca_data_agg %>% mutate(detection_rate = tp_rate/fp_rate) %>% filter(fp_rate>0.95) %>% sample_n(10)

In [ ]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2), panel.grid.major.y = element_line()))

## AgeSex + Metabolomics vs PANEL

In [ ]:
endpoints_plot = c(
         "M_type_2_diabetes", #s
                  "M_all_cause_dementia", #s
                  "M_heart_failure") #s

scores_plot = c(
    "Treat All", 
    "Treat None",
    "COX_Age+Sex",
    "DS_Age+Sex+Metabolomics",
    "DS_PANELnoblood+Metabolomics",
    "COX_ASCVD",
    "COX_PANEL",
    "DS_PANEL+Metabolomics"
)

In [ ]:
calibration_df = predictions_tte %>% 
    filter(endpoint %in% endpoints_plot) %>% 
    mutate(score = glue("{module}_{features}")) %>% 
    filter(score %in% scores_plot) %>% 
    select(eid, endpoint, score, Ft_10, event, event_time) %>%
    group_by(endpoint, score) %>% mutate(risk_group = ntile(Ft_10, 10), 
                                         event_10 = case_when(event == 0 ~ 0,
                                                              ((event==1)&(event_time>10)) ~ 0,
                                                              ((event==1)&(event_time<=10)) ~ 1
                                                             )
                                        ) %>% 
    group_by(endpoint, score, risk_group) %>% 
    mutate(mean_risk = mean(Ft_10), mean_events = mean(event_10)) %>%
    mutate(endpoint = factor(endpoint, levels=endpoints_plot))

In [ ]:
library(lemon)

In [ ]:
plots_cal = c()
for (endpoint in endpoints_plot){
    temp = calibration_df %>% filter(as.character(endpoint) == !!endpoint)
    endpoint_label = endpoint_map[[endpoint]]
    plots_cal[[endpoint]] = 
        ggplot(temp, aes(x=mean_risk*100, y=mean_events*100, color=score, size=score, linetype=score)) + 
        geom_abline(intercept=0, slope=1, color="#FA8334", alpha=1, size=0.5)+    
        #geom_point() + 
        geom_line(alpha=0.7)+
        labs(title=endpoint_label, x="Predicted Risk [%]", y="Observed Event Rate [%]") +
        scale_color_manual(values=color_map)+
        scale_linetype_manual(values=type_map)+
        scale_size_manual(values=size_map)+
        coord_cartesian(xlim=c(0, NA), ylim=c(0, NA))+
        theme(legend.position="none", axis.title = element_text())
    #print(plotsA[[endpoint]])
    flush.console()
    }

In [ ]:
fig5_cal = wrap_plots(plots_cal, ncol=3)
plot_width=8.25; plot_height=3.25; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
fig5_cal

In [ ]:
met_map = c("High"="#023768", "Mid"="#4F8EC1", "Low"="#7AC6FF")

In [ ]:
scores_plot = c(
    "Treat All", 
    "Treat None",
    "COX_Age+Sex",
    "DS_Age+Sex+Metabolomics",
    "DS_PANELnoblood+Metabolomics",
    "COX_ASCVD",
    "COX_PANEL"
)

plotsA = c()
for (endpoint in endpoints_plot){
    temp = dca_data[[endpoint]] %>% mutate(st_net_benefit = st_net_benefit*100) %>% mutate(st_net_benefit = case_when((label == "COX_Age+Sex")&is.na(st_net_benefit) ~ 0, TRUE ~ st_net_benefit))
    if (min((temp %>% filter(label=="DS_PANEL+Metabolomics"))$st_net_benefit, na.rm=TRUE)>0){
        threshold_max = (temp %>% filter(label=="DS_PANEL+Metabolomics") %>% slice_min(st_net_benefit))$threshold
        }
    else{
        threshold_max = min((temp %>% filter(label=="DS_PANEL+Metabolomics") %>% filter(st_net_benefit<0))$threshold, na.rm=TRUE)
        }
    endpoint_label = endpoint_map[[endpoint]]
        temp_plot = temp %>% filter(label %in% scores_plot)
    temp_plot_wide = temp_plot %>%  select(threshold, label, st_net_benefit) %>% pivot_wider(names_from="label", values_from="st_net_benefit")
    plotsA[[endpoint]] = 
        ggplot(temp_plot)+
          labs(title=endpoint_label, x="Threshold Probability [%]", y="Standardized Net Benefit [%]")+
        geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`COX_Age+Sex`, ymax=`DS_Age+Sex+Metabolomics`), fill=colors_dict$pastel$green$mid, alpha=0.5, colour = NA)+
        geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`DS_Age+Sex+Metabolomics`, ymax=`DS_PANELnoblood+Metabolomics`), fill="#023768", alpha=0.5, colour = NA)+
        geom_line(data=temp_plot, mapping=aes(x=threshold, y=st_net_benefit, color=label, linetype=label, size=label)) +       
        coord_cartesian(xlim=c(0, threshold_max), ylim=c(0, max(temp$st_net_benefit, na.rm=TRUE)))+
          scale_x_continuous(labels = function(x){x*100})+#labels = scales::percent_format(accuracy=1))+
          scale_color_manual(values=color_map)+
          scale_linetype_manual(values=type_map)+
    scale_size_manual(values=size_map)+
        theme(legend.position="none")
    #print(plotsA[[endpoint]])
    flush.console()
    }

In [ ]:
scores_plot = c(
    "Treat All", 
    "Treat None",
      "COX_Age+Sex",
      "COX_ASCVD",
    "COX_PANEL",
    "DS_PANEL+Metabolomics"
)

plotsB = c()
for (endpoint in endpoints_plot){
    temp = dca_data[[endpoint]] %>% mutate(st_net_benefit = st_net_benefit*100) 
    if (min((temp %>% filter(label=="DS_PANEL+Metabolomics"))$st_net_benefit, na.rm=TRUE)>0){
        threshold_max = (temp %>% filter(label=="DS_PANEL+Metabolomics") %>% slice_min(st_net_benefit))$threshold
        }
    else{
        threshold_max = min((temp %>% filter(label=="DS_PANEL+Metabolomics") %>% filter(st_net_benefit<0))$threshold, na.rm=TRUE)
        }
    endpoint_label = endpoint_map[[endpoint]]
        temp_plot = temp %>% filter(label %in% scores_plot)
    temp_plot_wide = temp_plot %>%  select(threshold, label, st_net_benefit) %>% pivot_wider(names_from="label", values_from="st_net_benefit")
    plotsB[[endpoint]] = 
        ggplot(data=temp_plot) + 
             labs(title=endpoint_label, x="Threshold Probability [%]", y="Standardized Net Benefit [%]")+
            
            geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=COX_PANEL, ymax=`DS_PANEL+Metabolomics`), fill="firebrick", alpha=0.7, colour = NA)+
            geom_line(data=temp_plot, mapping=aes(x=threshold, y=st_net_benefit, color=label, linetype=label, size=label)) +         
            coord_cartesian(xlim=c(0, threshold_max), ylim=c(0, max(temp$st_net_benefit, na.rm=TRUE)))+
            scale_x_continuous(labels = function(x){x*100})+#scales::percent_format(accuracy=1))+
            scale_color_manual(values=color_map)+
            scale_linetype_manual(values=type_map)+
            scale_size_manual(values=size_map)+
            theme(legend.position="none")
    #print(plotsB[[endpoint]])
    flush.console()
    }

In [ ]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2), panel.grid.major.y = element_line()))

In [ ]:
zip <- function(...) {
  mapply(list, ..., SIMPLIFY = TRUE)
}
plots = c(plotsA, plotsB)
fig5_dca = wrap_plots(plots, ncol=3)
plot_width=8.25; plot_height=6.5; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
fig5_dca

In [ ]:
fig5 = (fig5_cal / fig5_dca) +  plot_layout(height = c(1, 2.5)) & theme(plot.title = element_text(hjust = 0.5))
plot_width=8.25; plot_height=9; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
fig5

In [ ]:
library(gt)
plot_name = "Figures_5_ABC_ClinicalUtility"
fig5 %>% ggsave(filename=glue("outputs/{plot_name}.pdf"), device="pdf", width=plot_width, height=plot_height, dpi=plot_dpi)

In [ ]:
library(gt)
plot_name = "Figures_5_ABC_ClinicalUtility"
fig5 %>% ggsave(filename=glue("outputs/{plot_name}.png"), device="png", width=plot_width, height=plot_height, dpi=plot_dpi)